In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sampleSubmission.csv', 'test.csv', 'train.csv']


In [2]:
train_data = pd.read_csv("../input/train.csv")
train_data.head()
print("train_data shape: ", train_data.shape)

train_data shape:  (878049, 9)


In [3]:
target = train_data["Category"].unique()
print(target.shape)
target

(39,)


array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING',
       'SEX OFFENSES NON FORCIBLE', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'TREA', 'RECOVERED VEHICLE', 'PORNOGRAPHY/OBSCENE MAT'],
      dtype=object)

In [4]:
#df.drop(df.columns[[0, 1, 3]], axis=1)
X = train_data.drop(train_data.columns[[1, 2, 5, 6]], axis = 1)
X.head()

,Dates,DayOfWeek,PdDistrict,X,Y
0,2015-05-13 23:53:00,Wednesday,NORTHERN,-122.425892,37.774599
1,2015-05-13 23:53:00,Wednesday,NORTHERN,-122.425892,37.774599
2,2015-05-13 23:33:00,Wednesday,NORTHERN,-122.424363,37.800414
3,2015-05-13 23:30:00,Wednesday,NORTHERN,-122.426995,37.800873
4,2015-05-13 23:30:00,Wednesday,PARK,-122.438738,37.771541


In [5]:
y = train_data.iloc[:, 1]
y.head()

0          WARRANTS
1    OTHER OFFENSES
2    OTHER OFFENSES
3     LARCENY/THEFT
4     LARCENY/THEFT
Name: Category, dtype: object

In [6]:
def preprocess_data(dataset):
    dataset['Dates'] = pd.to_datetime(dataset['Dates'])
    dataset['Month'] = dataset.Dates.apply(lambda x: x.month)
    dataset['Day'] = dataset.Dates.apply(lambda x: x.day)
    dataset['Hour'] = dataset.Dates.apply(lambda x: x.hour)
    dataset['Minute'] = dataset.Dates.apply(lambda x: x.minute)
    dataset = dataset.drop('Dates', 1)
    
    dataset = pd.get_dummies(data=dataset, columns=['DayOfWeek', 'PdDistrict'])
    return dataset

In [7]:
X = preprocess_data(X)
X.head()

,X,Y,Month,Day,Hour,Minute,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,...,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
0,-122.425892,37.774599,5,13,23,53,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,-122.425892,37.774599,5,13,23,53,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,-122.424363,37.800414,5,13,23,33,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,-122.426995,37.800873,5,13,23,30,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,-122.438738,37.771541,5,13,23,30,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
from sklearn.preprocessing import LabelEncoder
y = y.to_frame()
le = LabelEncoder()
y["Category"] = le.fit_transform(y["Category"])
print(y.head())

   Category
0        37
1        21
2        21
3        16
4        16


In [9]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'ARSON': 0, 'ASSAULT': 1, 'BAD CHECKS': 2, 'BRIBERY': 3, 'BURGLARY': 4, 'DISORDERLY CONDUCT': 5, 'DRIVING UNDER THE INFLUENCE': 6, 'DRUG/NARCOTIC': 7, 'DRUNKENNESS': 8, 'EMBEZZLEMENT': 9, 'EXTORTION': 10, 'FAMILY OFFENSES': 11, 'FORGERY/COUNTERFEITING': 12, 'FRAUD': 13, 'GAMBLING': 14, 'KIDNAPPING': 15, 'LARCENY/THEFT': 16, 'LIQUOR LAWS': 17, 'LOITERING': 18, 'MISSING PERSON': 19, 'NON-CRIMINAL': 20, 'OTHER OFFENSES': 21, 'PORNOGRAPHY/OBSCENE MAT': 22, 'PROSTITUTION': 23, 'RECOVERED VEHICLE': 24, 'ROBBERY': 25, 'RUNAWAY': 26, 'SECONDARY CODES': 27, 'SEX OFFENSES FORCIBLE': 28, 'SEX OFFENSES NON FORCIBLE': 29, 'STOLEN PROPERTY': 30, 'SUICIDE': 31, 'SUSPICIOUS OCC': 32, 'TREA': 33, 'TRESPASS': 34, 'VANDALISM': 35, 'VEHICLE THEFT': 36, 'WARRANTS': 37, 'WEAPON LAWS': 38}


In [10]:
X.head()

,X,Y,Month,Day,Hour,Minute,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,...,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
0,-122.425892,37.774599,5,13,23,53,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,-122.425892,37.774599,5,13,23,53,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,-122.424363,37.800414,5,13,23,33,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,-122.426995,37.800873,5,13,23,30,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,-122.438738,37.771541,5,13,23,30,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
print(type(X_train), type(y_train))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 10, n_estimators = 256)
rf.fit(X_train.values, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=256, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
y_pred = rf.predict(X_test)
y_pred.shape
y_test.shape

from sklearn.metrics import accuracy_score
print ("Train Accuracy: ", accuracy_score(y_train, rf.predict(X_train)))
print ("Test Accuracy: ", accuracy_score(y_test, y_pred))

Train Accuracy:  0.26303778691103424
Test Accuracy:  0.25865839075223507


In [15]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns  
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm)
sns.heatmap(cm, cmap="YlGnBu", square=True)

In [16]:
print(classification_report(y_test, y_pred)) 

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       306
          1       0.24      0.00      0.00     15354
          2       0.00      0.00      0.00        82
          3       0.00      0.00      0.00        48
          4       0.00      0.00      0.00      7344
          5       0.00      0.00      0.00       799
          6       0.00      0.00      0.00       478
          7       0.31      0.29      0.30     10996
          8       0.00      0.00      0.00       842
          9       0.00      0.00      0.00       230
         10       0.00      0.00      0.00        52
         11       0.00      0.00      0.00        98
         12       0.00      0.00      0.00      2127
         13       0.00      0.00      0.00      3283
         14       0.00      0.00      0.00        35
         15       0.00      0.00      0.00       466
         16       0.28      0.84      0.42     34711
         17       0.00      0.00      0.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
test_data = pd.read_csv('../input/test.csv')
test_data.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [18]:
test_data = preprocess_data(test_data)
test_data.head()

,Id,Address,X,Y,Month,Day,Hour,Minute,DayOfWeek_Friday,DayOfWeek_Monday,...,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
0,0,2000 Block of THOMAS AV,-122.399588,37.735051,5,10,23,59,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,3RD ST / REVERE AV,-122.391523,37.732432,5,10,23,51,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,2000 Block of GOUGH ST,-122.426002,37.792212,5,10,23,50,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,4700 Block of MISSION ST,-122.437394,37.721412,5,10,23,45,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4,4700 Block of MISSION ST,-122.437394,37.721412,5,10,23,45,0,0,...,0,0,1,0,0,0,0,0,0,0


In [19]:
test_data = test_data.drop('Id', 1)
test_data = test_data.drop('Address', 1)

In [20]:
test_data.head()

,X,Y,Month,Day,Hour,Minute,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,...,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
0,-122.399588,37.735051,5,10,23,59,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,-122.391523,37.732432,5,10,23,51,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,-122.426002,37.792212,5,10,23,50,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,-122.437394,37.721412,5,10,23,45,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,-122.437394,37.721412,5,10,23,45,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [21]:
y_pred_proba = rf.predict_proba(test_data)

In [22]:
colmn = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE","DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING","FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL","OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY","SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE","SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]
result = pd.DataFrame(y_pred_proba, columns=colmn)

result.to_csv(path_or_buf="rf_predict.csv",index=True, index_label = 'Id')